# 提取SV断点图
每一页有四个SV断点图，其坐标确定方法为：x坐标以染色体区间的两端为准，y坐标以上下两条界线为准;  
每一页内的顺序为，先左上 -> 右上 -> 左下 -> 右下
* 此前我用原PDF文件的子集形成一个新的PDF文件，来确定SV图像的坐标；但当应用于原PDF文件时，SV图像的坐标却发生了变化
* 建议：根据原PDF文件转换后的图片，确定SV坐标，更改coordinates以及w, h

In [15]:
from pdf2image import convert_from_path
import cv2
import os

# 将PDF文件转换为图片
def convert_pdf_to_images(pdf_path, output_dir, dpi=300):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    pages = convert_from_path(pdf_path, dpi=dpi)

    for i, page in enumerate(pages):
        page_filename = f"{output_dir}/page_{i+1}.png"
        page.save(page_filename, "PNG")
        print(f"Saved page as image: {page_filename}")

# 根据手动确定的SV图像坐标，为每一页切割出4个SV断点图
def extract_sv_image_from_page(image_path, output_dir, coordinates):
    img = cv2.imread(image_path)

    for idx, (x, y) in enumerate(coordinates):
        w, h = 1050, 380 # 切割的宽度和高度
        cropped_img = img[y:y+h, x:x+w]

        # 检查裁剪后的图像尺寸，如果不是1050x320则进行尺寸调整
        target_width, target_height = 1050, 320
        if cropped_img.shape[1] != target_width or cropped_img.shape[0] != target_height:
            print(f"调整图像尺寸从 {cropped_img.shape[1]}x{cropped_img.shape[0]} 到 {target_width}x{target_height}")
            cropped_img = cv2.resize(cropped_img, (target_width, target_height))

        output_filename = f"{output_dir}/{os.path.splitext(os.path.basename(image_path))[0]}_part_{idx+1}.png"
        cv2.imwrite(output_filename, cropped_img)
        print(f"Saved cropped SV image: {output_filename}")


# 设置文件路径
pdf_path = "/Users/xurui/back_up_unit/天津大学文件/本科毕设相关/Article/ShatterSeek_data/NG_Supplementary_db_4.pdf"
page_output_dir = "/Volumes/T7-shield/CS-Bachelor-Thesis/CNN_model/pdf_convert"
image_output_dir = "/Volumes/T7-shield/CS-Bachelor-Thesis/CNN_model/SV_graph.PCAWG/SV_graph.dataset4"


In [ ]:
# 将PDF文件转换为图片
convert_pdf_to_images(pdf_path, page_output_dir)

为每一个页面切除4个SV图像  
基于每一页的dppi为300的情况
* high confidence 坐标 h,w = 1050, 320  
coordinates = [
    (71, 150),
    (1290, 150),
    (71, 2005),
    (1290, 2005)
]
* low confidence 坐标 h,w = 1050, 320  
coordinates = [
    (90，146),
    (1290, 146),
    (90, 1972),
    (1290, 1972)
]
* dataset 3 坐标 h, w = 1050, 370  
coordinates = [
    (85, 170),
    (1300 ,170),
    (85, 1980),
    (1300, 1980)
]
* dataset 4 坐标 h, w = 1050, 380
coordinates = [
    (80, 170),
    (1290, 170),
    (80, 2028),
    (1290, 2028)
]


In [ ]:
# 使用脚本 get_coordinates.py 获取坐标
coordinates = [
    (80, 170),
    (1290, 170),
    (80, 2028),
    (1290, 2028)
]

if not os.path.exists(image_output_dir):
    os.makedirs(image_output_dir)

for page_file in os.listdir(page_output_dir):
    if page_file.endswith(".png"):
        page_path = os.path.join(page_output_dir, page_file)
        extract_sv_image_from_page(page_path, image_output_dir, coordinates)

# 提取文本信息
使用 extract_text_info_from_PDF.py 实现，提取样本名，Position, Oscillating CN （2 and 3 states） CN segments
* 需要注意的地方：
    * 为了避免OCR图片带来的错误，选择直接从PDF文件中提取文本部分
    * 页面之间四个染色体破裂事件的分布，并不是完全一样的（_都是左上角的事件，文本部分坐标kennel有细微差别_），所以划分区间的时候需要注意，可以稍微大一些
    * 算法中匹配样本名时发现，PCAWG原始表格中短横线 (-) 有多种类型，需要额外注意！
* 实现的算法：
    1. 先将每一页的文本都提取出来，包括文本和坐标（放在字典里），坐标用于划分到不同的事件中
    2. 用其他脚本，手动确定每个事件的大概范围，制作文字部分区间
    3. 同一文字区间的文本视为同一个事件的信息
    4. 匹配所需要的信息（例如样本名，POS），方法比较复杂，不同信息提取方式也略有不同
        * 样本名是使用PCAWG提供表格中 donor_idx 进行匹配  


# 验证提取的文本信息中有无缺失值

对于提取PDF时缺失的值，根据PCAWG的表格中的数据进行补充  
使用 *detect_missing_values.py* 脚本进行判断 
PCAWG表格和我们提取的表格之间，列的映射关系：
* CN segments -> cn_segments
* Nb. oscillating.CN -> CN_2_states
* Nb. oscillaring CN 3 states -> CN_3_states
* Chr + Start + End -> position
* donor_idx -> sample_name
    * 提取PDF中 sample_name 时出现了 (-) 不匹配PCAWG表格中 (-) 的问题，故提取信息的表格中使用的sample_name不是直接从PDF中提取的，而是使用PCAWG表格中对应的 donor_idx

### high confidence results 
Total rows in input file: 1136  
Rows with missing values: 21  
Missing values by column:  
  position: 21 missing values  
  
即只存在position列没有提取成功的情况 [经确认，是正则匹配时忽略了X染色体的情况，修改后已无缺失值]  

### low confidence results
经过检测没有缺失值

### Dataset 3 results
Total rows in input file: 816  
Rows with missing values: 216 
Missing values by column:  
  position: 216 missing values  
  oscillating_cn_2&3_states: 5 missing values  
  cn_segments: 216 missing values  
经过检查，检测不成功的样本是 position 显示为染色体号，没有区间；并且 CNV 信息也只有在两个状态震荡的值，后续会被舍弃

### Dataset 4 Results
没有缺失值

# 使用PCAWG人工审查结果打标签

PCAWG中有注释的条目只有两种类型: False negative; manually included 和 False positive; manually removed  
* High Confidence 中没有任何匹配项
* Low Confidence 中没有任何匹配
* Dataset3 有162个匹配项 -- 已经去除了 missing values 的情况
    * 161 个 False Positive
    * 1 个 False Negative (C0052 chr6) 
* Dataset4 有两个匹配项
    * 2个 False negative; manually included
    * donor_idx: 17dffffc-65d6-4209-9075-18a441001f0f Chr: 12
    * donor_idx: c184c3ca-7ad3-4202-b108-cb9fd5f5d947 Chr: 12


In [17]:
import pandas as pd
import re
import os

def split_position(position_str):
    """
    将位置字符串(Chr:Start-End)拆分为三个组件
    """
    if not isinstance(position_str, str) or not position_str:
        return None, None, None
        
    # 使用正则表达式匹配位置格式
    match = re.match(r'([0-9XY]+):(\d+)[−-](\d+)', position_str)
    if match:
        chr_val = match.group(1)
        start_val = int(match.group(2))
        end_val = int(match.group(3))
        return chr_val, start_val, end_val
    else:
        return None, None, None

def process_combined_events(tsv_path):
    """
    处理combined_events.tsv文件，拆分position列
    """
    try:
        # 读取TSV文件
        df = pd.read_csv(tsv_path, sep='\t')
        
        # 检查position列是否存在
        if 'position' not in df.columns:
            print(f"错误: {tsv_path}中没有'position'列")
            return None
            
        # 创建新列存储拆分结果
        df['Chr'] = None
        df['Start'] = None
        df['End'] = None
        
        # 拆分每一行的position
        failure_count = 0
        for idx, row in df.iterrows():
            chr_val, start_val, end_val = split_position(row['position'])
            if chr_val is None:
                failure_count += 1
                print(f"警告: 无法拆分position值 '{row['position']}' (行 {idx+2})")
            else:
                df.at[idx, 'Chr'] = chr_val
                df.at[idx, 'Start'] = start_val
                df.at[idx, 'End'] = end_val
        
        print(f"位置拆分完成，成功: {len(df) - failure_count}, 失败: {failure_count}")
        return df
    
    except Exception as e:
        print(f"处理{tsv_path}时出错: {str(e)}")
        return None

def find_matching_records(combined_df, abnormal_path):
    """
    查找abnormal_calls_from_PCAWG.tsv中与combined_events.tsv匹配的记录
    """
    try:
        # 读取abnormal文件
        abnormal_df = pd.read_csv(abnormal_path, sep='\t')
        
        # 检查必要的列是否存在
        required_cols = ['donor_idx', 'Chr', 'Start', 'End']
        for col in required_cols:
            if col not in abnormal_df.columns:
                print(f"错误: {abnormal_path}中缺少'{col}'列")
                return
        
        # 确保数据类型一致
        combined_df['Chr'] = combined_df['Chr'].astype(str)
        combined_df['Start'] = pd.to_numeric(combined_df['Start'], errors='coerce')
        combined_df['End'] = pd.to_numeric(combined_df['End'], errors='coerce')
        
        abnormal_df['Chr'] = abnormal_df['Chr'].astype(str)
        abnormal_df['Start'] = pd.to_numeric(abnormal_df['Start'], errors='coerce')
        abnormal_df['End'] = pd.to_numeric(abnormal_df['End'], errors='coerce')
        
        # 创建字典用于统计comment频率
        comment_counts = {}
        
        # 遍历abnormal文件中的每一行
        match_count = 0
        for idx, abnormal_row in abnormal_df.iterrows():
            # 查找匹配记录
            matches = combined_df[
                (combined_df['sample_name'] == abnormal_row['donor_idx']) &
                (combined_df['Chr'] == abnormal_row['Chr']) &
                (combined_df['Start'] == abnormal_row['Start']) &
                (combined_df['End'] == abnormal_row['End'])
            ]
            
            if len(matches) > 0:
                match_count += 1
                print(f"\n匹配记录 #{match_count}:")
                print(f"  Abnormal记录 (行 {idx+2}):")
                print(f"    donor_idx: {abnormal_row['donor_idx']}")
                print(f"    Chr: {abnormal_row['Chr']}")
                print(f"    Start: {abnormal_row['Start']}")
                print(f"    End: {abnormal_row['End']}")
                
                # 打印comment列(如果存在)
                if 'comment' in abnormal_row:
                    comment = str(abnormal_row['comment'])
                    print(f"    comment: {comment}")
                    
                    # 统计comment频率
                    if comment in comment_counts:
                        comment_counts[comment] += 1
                    else:
                        comment_counts[comment] = 1
                
                for _, match in matches.iterrows():
                    print(f"  匹配的Combined记录:")
                    print(f"    sample_name: {match['sample_name']}")
                    print(f"    position: {match['position']}")
                    print(f"    Chr: {match['Chr']}")
                    print(f"    Start: {match['Start']}")
                    print(f"    End: {match['End']}")
                    if 'oscillating_cn_2&3_states' in match:
                        print(f"    oscillating_cn: {match['oscillating_cn_2&3_states']}")
                    if 'cn_segments' in match:
                        print(f"    cn_segments: {match['cn_segments']}")
        
        print(f"\n总计找到 {match_count} 条匹配记录")
        
        # 打印comment频率统计
        if comment_counts:
            print("\nComment频率统计:")
            # 按频率降序排序
            sorted_comments = sorted(comment_counts.items(), key=lambda x: x[1], reverse=True)
            for comment, count in sorted_comments:
                print(f"  '{comment}': {count}次 ({count/match_count*100:.1f}%)")
        
    except Exception as e:
        print(f"处理{abnormal_path}时出错: {str(e)}")

def main():
    # 设置文件路径
    base_dir = "/Volumes/T7-shield/CS-Bachelor-Thesis/CNN_model"
    combined_path = os.path.join(base_dir, "SV_graph.PCAWG/CNV_info_from_PDF/dataset4_info/combined_events.tsv")
    abnormal_path = os.path.join(base_dir, "PCAWG-related/abnormal_calls_from_PCAWG.tsv")
    output_path = os.path.join(base_dir, "SV_graph.PCAWG/CNV_info_from_PDF/dataset4_info/combined_events_split.tsv")

    # 检查文件是否存在
    for path in [combined_path, abnormal_path]:
        if not os.path.exists(path):
            print(f"错误: 文件 {path} 不存在")
            return
    
    # 第一步: 处理combined_events.tsv，拆分position列
    print("=== 第一步: 处理combined_events.tsv ===")
    combined_df = process_combined_events(combined_path)
    if combined_df is None:
        return
    
    # 保存拆分后的结果（可选）
    combined_df.to_csv(output_path, sep='\t', index=False)
    print(f"拆分结果已保存到 {output_path}")
    
    # 第二步: 查找匹配记录
    print("\n=== 第二步: 查找匹配记录 ===")
    find_matching_records(combined_df, abnormal_path)

if __name__ == "__main__":
    main()

=== 第一步: 处理combined_events.tsv ===
位置拆分完成，成功: 264, 失败: 0
拆分结果已保存到 /Volumes/T7-shield/CS-Bachelor-Thesis/CNN_model/SV_graph.PCAWG/CNV_info_from_PDF/dataset4_info/combined_events_split.tsv

=== 第二步: 查找匹配记录 ===

匹配记录 #1:
  Abnormal记录 (行 69):
    donor_idx: 17dffffc-65d6-4209-9075-18a441001f0f
    Chr: 12
    Start: 57486350.0
    End: 69728850.0
    comment: False negative; manually included
  匹配的Combined记录:
    sample_name: 17dffffc-65d6-4209-9075-18a441001f0f
    position: 12:57486350−69728850
    Chr: 12
    Start: 57486350
    End: 69728850
    oscillating_cn: 3, 6
    cn_segments: 39

匹配记录 #2:
  Abnormal记录 (行 76):
    donor_idx: c184c3ca-7ad3-4202-b108-cb9fd5f5d947
    Chr: 12
    Start: 57802243.0
    End: 59971140.0
    comment: False negative; manually included
  匹配的Combined记录:
    sample_name: c184c3ca-7ad3-4202-b108-cb9fd5f5d947
    position: 12:57802243−59971140
    Chr: 12
    Start: 57802243
    End: 59971140
    oscillating_cn: 3, 5
    cn_segments: 48

总计找到 2 条匹配记录

Comment